In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import math
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import sys
import os
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib
import hpc_reports
#
def running_mean(X,n=10):
    return (numpy.cumsum(numpy.insert(X,0,0))[n:] - numpy.cumsum(numpy.insert(X,0,0))[:-n])/n
#
#data_file_name = 'data/sacct_serc_20200622.out'
#data_file_name = 'data/sacct_serc_20200724.pkl'

In [3]:
# load_pickle=True
# pkl_name = "{}.pkl".format(os.path.splitext(data_file_name)[0])
# #
# if load_pickle:
#     with open(pkl_name, 'rb') as fin:
#         sacct_mazama=pickle.load(fin)
#     #
# else:
#     sacct_mazama = hpc_lib.SACCT_data_handler(data_file_name=data_file_name)
#     #
    
#     with open(pkl_name, 'wb') as fout:
#             #out_pkl = pickle.dump(sacct_demo.jobs_summary, fout)
#             out_pkl = pickle.dump(sacct_mazama, fout)

In [4]:
# n_cpus:
n_cpus = 1
if 'SLURM_JOB_ID' in os.environ.keys():
    # running in SLURM...
    if 'SLURM_JOB_CPUS_PER_NODE' in os.environ.keys():
        n_cpus = max( int(os.environ['SLURM_JOB_CPUS_PER_NODE']), 1)
    else:
        n_cpus = 1
else:
    n_cpus = min(8, mpp.cpu_count())
#
# or specify here:
#n_cpus = 4
#
print('** n_cpus: ', n_cpus)

** n_cpus:  1


In [5]:
#end_dtm = dtm.datetime.now()+dtm.timedelta(days=1)
end_dtm = dtm.datetime(2022,2,28)
sacct_h5_file = f'sacct_data/serc_sacct_{end_dtm.year:04}{end_dtm.month:02}{end_dtm.day:02}.h5'
#start_date = '2021-04-15'
#end_date   = '2021-05-15'
end_date = end_dtm.date()
start_date = end_date - dtm.timedelta(days=180)
print('*** dates: {} - {}'.format(start_date, end_date))
#
# NOTE: additional options can be passed in the more_options=[] arrary, or just as sacct_{option-name}={val}
#
if os.path.isfile(sacct_h5_file):
    print('*** Creating SACCT object from HDF5 {}'.format(sacct_h5_file))
    SACCT_obj = hpc_lib.SACCT_data_from_h5(sacct_h5_file, keep_raw_data=False, n_cpu=n_cpus)
    #
    
else:
    print('*** Fetching SACCT data directly')
    SACCT_obj = hpc_lib.SACCT_data_direct(group=None, partition='serc', start_date=str(start_date),
                                          n_cpu=n_cpus,
                                          end_date=str(end_date), keep_raw_data=False)
    print(f'** writing HDF5: {sacct_h5_file}')
    SACCT_obj.write_hdf5(sacct_h5_file)
    #
#
print('** ', SACCT_obj.jobs_summary.dtype.names)

*** dates: 2021-09-01 - 2022-02-28
*** Creating SACCT object from HDF5 sacct_data/serc_sacct_20220228.h5
**  ('index', 'User', 'Group', 'GID', 'JobName', 'JobID', 'JobIDRaw', 'Partition', 'State', 'Timelimit', 'NCPUS', 'NNodes', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed', 'SystemCPU', 'UserCPU', 'TotalCPU', 'NTasks', 'CPUTimeRAW', 'Suspended', 'ReqTRES', 'AllocTRES', 'JobID_parent')


In [6]:
max_submit, max_start = [mpd.num2date(numpy.nanmax(SACCT_obj.jobs_summary[cl])) for cl in ['Start', 'Submit'] ]
print('*** max_submit: {}, max_start: {}'.format(*[mpd.num2date(numpy.nanmax(SACCT_obj.jobs_summary[cl]))
                                                   for cl in ['Start', 'Submit'] ]))
print('** cols: ', SACCT_obj.jobs_summary.dtype.names)

*** max_submit: 2022-02-28 16:07:17+00:00, max_start: 2022-02-27 23:37:16+00:00
** cols:  ('index', 'User', 'Group', 'GID', 'JobName', 'JobID', 'JobIDRaw', 'Partition', 'State', 'Timelimit', 'NCPUS', 'NNodes', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed', 'SystemCPU', 'UserCPU', 'TotalCPU', 'NTasks', 'CPUTimeRAW', 'Suspended', 'ReqTRES', 'AllocTRES', 'JobID_parent')


In [7]:
# NOTE: add this to the __init__. would be nice to refactor the code a bit to better handle inheritance.
if not 'chunk_size' in SACCT_obj.__dict__.keys():
    print('*** assigning chunk_size: ', chunk_size)
    SACCT_obj.chunk_size=1000

In [ ]:
#
cpu_usage = SACCT_obj.active_jobs_cpu()
#
bin_size=7
cpu_weekly = SACCT_obj.active_jobs_cpu(bin_size=bin_size, t_min=None)

*** DEBUG: 5000, None


In [ ]:
print('** ', len(cpu_usage))
print('** \n', cpu_usage[0:10])

In [ ]:
fg = plt.figure(figsize=(10,10))
ax1 = fg.add_subplot('211')
ax2 = fg.add_subplot('212', sharex=ax1)
for ax in (ax1, ax2):
    ax.grid()
#
ax1.plot(cpu_usage['time'], cpu_usage['N_jobs'], ls='-', lw=2., marker='')
ax1.plot(cpu_weekly['time'], cpu_weekly['N_jobs'], ls='-', lw=2., marker='.')
#
ax2.plot(cpu_usage['time'], cpu_usage['N_cpu'], ls='-', lw=2., marker='')
ax2.plot(cpu_weekly['time'], cpu_weekly['N_cpu'], ls='-', lw=2., marker='.')
#

ax1.set_title('Jobs', size=16)
ax1.set_ylabel('$N_{jobs}$', size=16)
#
ax2.set_title('CPUs', size=16)
ax2.set_ylabel('$N_{CPU}$', size=16)

fg.canvas.draw()
#
# set ax3 labels to dates:
# now format the datestrings...
# fix_to_ascii(s)
lbls = [hpc_lib.simple_date_string(mpd.num2date(float(hpc_lib.fix_to_ascii(s.get_text()))) ) for s in ax1.get_xticklabels()]
#print('*** ', lbls)
#
ax2.set_xticklabels(lbls)



In [ ]:
# def time_bin_aggregates(XY, bin_mod=24, qs=[.25, .5, .74]):
#     XY=numpy.array(XY)
#     if XY.shape[0]==2:
#         X = XY[0,:]
#         Y = XY[1:]
#     else:
#         X = XY[:,0]
#         Y = XY[:,1]
#     #
#     #X_mod = ((X*bin_mod)%bin_mod).astype(int)
#     X_mod = ((X%1.)*bin_mod).astype(int)
#     #
#     stats_output=[]
#     for x in numpy.unique(X_mod):
#         ix = X_mod==x
#         this_Y = Y[ix]
#         stats_output += [numpy.append([x, numpy.mean(this_Y), numpy.std(this_Y)],
#                                       numpy.quantile(this_Y, qs))]
#     #
#     return numpy.core.records.fromarrays(numpy.array(stats_output).T, dtype=[('x', '>f8'), ('mean', '>f8'),
#                                                         ('stdev', '>f8')] + 
#                                          [('q_{}'.format(q), '>f8') for q in qs])
# #

    

In [ ]:
qs = [.5, .75, .95]
#
comp_vol_submit = SACCT_obj.get_submit_compute_vol_timeofday(qs=qs)
comp_vol_start = SACCT_obj.get_submit_compute_vol_timeofday(time_col='Start', qs=qs)
#
fg = plt.figure(figsize=(12,6))
ax1 = fg.add_subplot('121')
ax2 = fg.add_subplot('122')
ax1.grid()
ax2.grid()
#

# N = numpy.sum(comp_vol_submit['cpu-time'])
N = 1.
ax1.plot(comp_vol_submit['time'], comp_vol_submit['cpu-time']/N,
         ls='-', marker='o', lw=2., label='submit')
#print('*** ', numpy.sum(comp_vol_submit['cpu-time']/numpy.sum(comp_vol_submit['cpu-time'])))
#
# N = numpy.sum(comp_vol_start['cpu-time'])
N = 1 
ax1.plot(comp_vol_start['time'], comp_vol_start['cpu-time']/N,
         ls='-', marker='o', lw=2., label='start')
#print('*** ', numpy.sum(comp_vol_start['cpu-time']/N))
#
#N=numpy.sum(comp_vol_submit['cpus'])
N=1.
ax2.plot(comp_vol_submit['time'], comp_vol_submit['cpus']/N, ls='-', marker='o', lw=2., label='submit')

#N=numpy.sum(comp_vol_start['cpus'])
N=1
ax2.plot(comp_vol_start['time'], comp_vol_start['cpus']/N, ls='-', marker='o', lw=2., label='start')
#
#for k,cl in enumerate(comp_vol_tod.dtype.names[2:]):
#    ax1.plot(comp_vol_tod['time'], comp_vol_tod[cl], ls='-', marker='o', lw=2., label='$q={}$'.format(qs[k]))
#    break
#
ax1.legend(loc=0, numpoints=1)
ax1.set_title('Compute Volume Requested, \n$N_{CPU} \cdot \Delta t_{limit}$')
ax1.set_xlabel('Hour of day')
ax2.set_ylabel('Compute Vol.')

ax2.set_title('CPUs Requested')
ax2.set_xlabel('Hour of day')
ax2.legend(loc=1, numpoints=1)

In [ ]:
fg = plt.figure(figsize=(10,10))
ax1 = fg.add_subplot(2,2,1)
ax2 = fg.add_subplot(2,2,2)
ax3 = fg.add_subplot(2,2,3)
ax4 = fg.add_subplot(2,2,4)
axs = [ax1, ax2, ax3, ax4]
[ax.grid() for ax in axs]
#
cpu_hourly = hpc_lib.time_bin_aggregates(XY=numpy.array([cpu_usage['time'], cpu_usage['N_cpu']]).T)
jobs_hourly = hpc_lib.time_bin_aggregates(XY=numpy.array([cpu_usage['time'], cpu_usage['N_jobs']]).T)

hh1 = ax1.hist(sorted(cpu_usage['N_jobs'])[0:int(1.0*len(cpu_usage))], bins=25, cumulative=False)
ax2.plot(jobs_hourly['x'], jobs_hourly['mean'], ls='-', marker='o')

hh3 = ax3.hist(cpu_usage['N_cpu'], bins=25)
ax4.plot(cpu_hourly['x'], cpu_hourly['mean'], ls='-', marker='o')

#ax1.set_ylim(-5., 200)
ax1.set_title('$N_{jobs}$ Histogrm', size=16)
ax2.set_title('Hour-of-day job counts', size=16)
ax3.set_title('$N_{cpu}$ Histogram', size=16)
ax4.set_title('Hour-of-day CPU counts', size=16)
#
plt.suptitle('Instantaneous Usage', size=16)

In [ ]:
weekly_hours = SACCT_obj.get_cpu_hours(bin_size=7, n_points=5000)

daily_hours = SACCT_obj.get_cpu_hours(bin_size=1, n_points=5000)

In [ ]:
fg = plt.figure(figsize=(10,8))
ax1 = plt.gca()
ax1.grid()
#
ax1.plot(weekly_hours['time'], weekly_hours['cpu_hours']/7., ls='-', marker='.', label='bins=7 day', zorder=11)
ax1.plot(daily_hours['time'], daily_hours['cpu_hours'], ls='-', marker='.', label='bins=1 day', zorder=5)
#
#ax1.plot( daily_hours['time'][0::(len(daily_hours['time'])-1)], numpy.ones(2)*14*24*24, ls='--', lw=3.)
#ax1.plot( daily_hours['time'][0::(len(daily_hours['time'])-1)], numpy.ones(2)*12*24*24, ls='--', lw=3.)
#
n_cpus_serc = 32*104 + 128*8 + 24*12
n_cpus_serc_gpus = n_cpus_serc + 128*6 + 24*2
ax1.plot( daily_hours['time'][0::(len(daily_hours['time'])-1)], numpy.ones(2)*24*n_cpus_serc, ls='--', lw=3.,
         label='CPU capacity')
#ax1.plot( daily_hours['time'][0::(len(daily_hours['time'])-1)], numpy.ones(2)*24*n_cpus_serc_gpus, ls='--', lw=3.)




#
fg.canvas.draw()
#
# set ax3 labels to dates:
# now format the datestrings...
lbls = [hpc_lib.simple_date_string(mpd.num2date(float(hpc_lib.fix_to_ascii(s.get_text()))) ) for s in ax1.get_xticklabels()]
#print('*** ', lbls)
#
ax1.set_xticklabels(lbls)
ax1.set_xlabel('Time $t$')
ax1.set_ylabel('Daily CPU hours')
ax1.set_title('Daily CPU hours')
#
ax1.legend(loc=0)

In [ ]:
print('** dtype:\n', SACCT_obj.jobs_summary.dtype)
#
#user_cols  = ['Timelimit', 'NCPUS', 'NNodes', 'TotalCPU','NTasks']
#user_types = ['>f8', '>i8', '>i8', '>f8', '>i8', '>f8']

user_cols = ['NCPUS', 'NNodes', 'Elapsed', 'Comp_Vol']
user_types = ['>i8', '>i8', '>f8', '>f8']
#
user_aggs = numpy.zeros( shape=[len(numpy.unique(SACCT_obj.jobs_summary['User'])),],
                        dtype={'names':user_cols, 'formats':user_types })
#
ix = {usr:numpy.where(SACCT_obj.jobs_summary['User']==usr)[0] 
      for usr in numpy.unique(SACCT_obj.jobs_summary['User'])}
#

In [ ]:
print('*** ', user_aggs.shape, user_aggs.dtype)
print('*** ', user_cols)
user_names = numpy.unique(SACCT_obj.jobs_summary['User'])
for k,user in enumerate(user_names):
    #print('** ', [numpy.sum(SACCT_obj.jobs_summary[cl][ix[user]]) for cl in user_cols])
    #
    vals = [numpy.sum( numpy.max([numpy.zeros(len(ix[user])), SACCT_obj.jobs_summary[cl][ix[user]]], axis=0) ) 
                      for cl in user_cols[:-1]]
    vals += [numpy.sum(SACCT_obj.jobs_summary['Elapsed'][ix[user]]*24. * SACCT_obj.jobs_summary['NCPUS'][ix[user]])]
    #print('** ix: ', ix[user])
    #vals = [numpy.sum( (SACCT_obj.jobs_summary[cl][ numpy.array(ix[user])] ) )
    #        for cl in user_cols]
    #print('** vals: ', vals)
    
    #user_aggs[user_cols][k] = tuple([numpy.sum(SACCT_obj.jobs_summary[cl][ix[user]]) for cl in user_cols])
    
    user_aggs[user_cols][k] = tuple( vals )
    #user_aggs[user_cols][k] = tuple(vals)



In [ ]:
# print('** user, {}'.format(user_cols ) )
# for nm,rw in zip(user_names, user_aggs):
#     print('** {}: {}, '.format(nm,rw))

In [ ]:
fg = plt.figure(figsize=(10,8))
ax = plt.gca()
ax.grid()
for k,usr in enumerate(user_names[numpy.argsort(user_aggs['Comp_Vol'])[::-1]]):
    cpuh = SACCT_obj.get_cpu_hours(bin_size=1.0, jobs_summary=SACCT_obj.jobs_summary[ix[usr]])
    ax.plot(cpuh['time'], cpuh['cpu_hours'], label=('{}: {}'.format(usr, numpy.sum(cpuh['cpu_hours']) )
                                                    if k<5 else None) )
ax.legend(loc=0)

In [ ]:
# time of day(-like) compute volume requests.

qs = [.5, .75, .95]
#
comp_vol_submit = SACCT_obj.get_submit_compute_vol_timeofday(qs=qs)
comp_vol_start = SACCT_obj.get_submit_compute_vol_timeofday(time_col='Start', qs=qs)
#
fg = plt.figure(figsize=(12,6))
ax1 = fg.add_subplot('121')
ax2 = fg.add_subplot('122')
ax1.grid()
ax2.grid()
#

ax1.plot(comp_vol_submit['time'], comp_vol_submit['cpu-time']/numpy.sum(comp_vol_submit['cpu-time']),
         ls='-', marker='o', lw=2., label='submit')
#print('*** ', numpy.sum(comp_vol_submit['cpu-time']/numpy.sum(comp_vol_submit['cpu-time'])))

ax1.plot(comp_vol_start['time'], comp_vol_start['cpu-time']/numpy.sum(comp_vol_start['cpu-time']),
         ls='-', marker='o', lw=2., label='start')
#print('*** ', numpy.sum(comp_vol_start['cpu-time']/numpy.sum(comp_vol_start['cpu-time'])))
#
ax2.plot(comp_vol_submit['time'], comp_vol_submit['cpus']/numpy.sum(comp_vol_submit['cpus']), ls='-', marker='o', lw=2., label='submit')
ax2.plot(comp_vol_start['time'], comp_vol_start['cpus']/numpy.sum(comp_vol_start['cpus']), ls='-', marker='o', lw=2., label='start')
#
#for k,cl in enumerate(comp_vol_tod.dtype.names[2:]):
#    ax1.plot(comp_vol_tod['time'], comp_vol_tod[cl], ls='-', marker='o', lw=2., label='$q={}$'.format(qs[k]))
#    break
#
ax1.legend(loc=0, numpoints=1)
ax1.set_title('Compute Volume Requested, \n$N_{CPU} \cdot \Delta t_{limit}$')
ax1.set_xlabel('Hour of day')
ax2.set_ylabel('Percent')

ax2.set_title('CPUs Requested')
ax2.legend(loc=1, numpoints=1)

#ax1.set_yscale('log')

In [ ]:
# compute some distributions:
print('*** ', SACCT_obj.jobs_summary.dtype.names)
run_times = SACCT_obj.get_run_times()

#
fg = plt.figure(figsize=(12,8))
ax1 = fg.add_subplot(2,2,1)
ax2 = fg.add_subplot(2,2,2)
#ax1a.set_yscale('log')
ax3 = fg.add_subplot(2,2,3)
ax4 = fg.add_subplot(2,2,4)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
#
hh = ax1.hist(run_times, bins=50, cumulative=False, density=True, log=True)
hh1c = ax2.hist(run_times, bins=50, cumulative=True, density=True, log=True, histtype='bar')
#hh1c = ax2.plot(sorted(run_times), numpy.array(numpy.linspace(1./len(run_times), 1.,
#                                                             len(run_times))), lw=3.0, zorder=11)
#
hh2 = ax3.hist(SACCT_obj.jobs_summary['NCPUS'], bins=50, density=True, log=True)
hh22 = ax4.hist(SACCT_obj.jobs_summary['NCPUS'], bins=50, density=True, log=True,
                 cumulative=True, histtype='bar', lw=3, zorder=11)
#hh22 = ax4.hist(sacct_mazama.jobs_summary['NCPUS'], bins=50, normed=True, log=True,
#                 cumulative=True, histtype='step', lw=3, zorder=11)
#ax1.plot(run_times, ls='', marker='.')
#print('*** ', run_times[0:20]*24, len(run_times), run_times.shape)
#
ax1.set_title('Run-time Distribution (days)')
ax3.set_title('NCPUs Distribution')
ax2.set_xlabel('Time (days)')
ax4.set_xlabel('CPUS $N_{cpus}$')
#
ax2.set_title("(Cumulative)")
#ax4.set_title("(Cumulative)")

#ax2.set_xlim(-1,800)

In [ ]:
print('** ', len(run_times), len(SACCT_obj.jobs_summary['NCPUS']))

In [ ]:
#ix_rt = numpy.argsort(run_times)
run_times_sorted = run_times.copy()
run_times_sorted.sort()
#
k_2 = numpy.searchsorted(run_times_sorted, 2.0)
k_7 = numpy.searchsorted(run_times_sorted, 7.0)
k_14 = numpy.searchsorted(run_times_sorted, 14.0)

#
# k_2 = numpy.searchsorted(run_times[ix_rt], 2.0)
# #k_7 = numpy.searchsorted(run_times[ix_rt], 7.0)
# k_7 = k_2 + numpy.searchsorted( (run_times[ix_rt])[k_2:], 7.0)
# k_14 = k_7 + numpy.searchsorted( (run_times[ix_rt])[k_7:], 14.0)
#
N=float(len(run_times))
print('*** quantiles for t=2,7,14 days: {}, {}, {}'.format(float(k_2)/N, k_7/N, k_14/N))
#
# percent of jobs that use N<24 cores (aka, can run on a single node). Note that these are especially
#. eligible for GCP.
N_24 = numpy.sum(SACCT_obj.jobs_summary['NCPUS']<25)
print('*** N_24/N={}'.format(N_24/N))


In [ ]:
wait_stats = SACCT_obj.get_wait_stats(qs=[.25, .5, .75, .9])
#
fg = plt.figure(figsize=(10,10))
ax1 = plt.gca()
ax1.grid()
#
ax1.plot(wait_stats['ncpus'], wait_stats['mean']*60.*24, ls='-', label='mean')
ax1.plot(wait_stats['ncpus'], wait_stats['median']*60.*24, ls='-', label='median')
#
ax1.set_ylabel('Wait time (minutes)')
ax1.set_xlabel('$N_{CPUS}$', size=16)
#ax1.set_ylim(-.1, .5)
ax1.set_yscale('log')
ax1.legend(loc=0)

In [ ]:
# if True:
#     with open(data_file_name, 'r') as fin:
#         header_rw = fin.readline()
#         print('*** ', header_rw)

#     headers = header_rw.split('|')
#     k_group = headers.index('Group')
#     k_gid = headers.index('GID')
#     #
    
#     with open(data_file_name, 'r') as fin:
#         for k,rw in enumerate(fin):
#             #if 'dunham' in rw: print('** ', rw)
#             #if 'dunham' in rw:
#             #    rws = rw.split('|')
#             #    print('** ', rws[k_group], rws[k_gid])
#             if 'Partition_Limit' in rw:
#                 print('** ', rw)
#                 k+=1
#                 if k>10: break



In [ ]:
print('** ',SACCT_obj.jobs_summary.dtype.names)
print('** ', mpd.num2date(SACCT_obj.jobs_summary['Submit'][-1]))

## Drafts of some reports
- We have a couple reports already made. Let's start with those.
- Even for those reports, we'll want to modify the inputs, for example to specify all Earth users/groups, not juust look at the `serc` partition.


In [ ]:
### Report(s)
#
# first, we need to make groups.
# groups who can access serc:
# (base) [myoder96@sh01-ln01 login ~]$ scontrol show partition serc | grep Allow
#   AllowGroups=sh_s-ees,sh_sysadm AllowAccounts=ALL AllowQos=normal,high_p,system
#
# so all permissions are consolidated to the sh_s-ees group
# to get users in a group:
# USERS=`getent passwd | awk -F: -v g=$(id -g $PI_SUNET) '$4==g {print $1}'`
#. but this does not seem to work for the sh_s-ees group
# we can use id to get secondary groups (we'll need to pares...), for example,
# $ (base) [myoder96@sh01-ln01 login ~/Codes/cees_scripts]$ id myoder96
# uid=362778(myoder96) gid=32264(ruthm) groups=99001(sh_users),98013(sh_sw-stata),98008(sh_sw-schrodinger),97002(sh_s-ees),97005(sh_s-hns),1007582(oak_p-cees-backup),1008811(oak_s-ees),32264(ruthm)
# this should do it:
# getent group sh_s-ees
# then maybe just parse in Python...

In [ ]:
# PI_groups = hpc_lib.get_PI_groups_from_groups(groups='sh_s-ees')
# #
# for ky,vl in PI_groups.items():
#     print('** {}, {}'.format(ky,vl))
#     #
# #
# SACCT = hpc_lib.SACCT_data_direct(partition='serc', start_date='2021-01-01', end_date='2021-06-03',
#                                   keep_raw_data=False)
# #
# rpt = hpc_lib.SACCT_groups_analyzer_report(Short_title='SERC HPC Analytics', 
#                 Full_title='HPC Analytics Breakdown for Stanford Earth, Sherlock SERC',
#                 out_path='output/SERC_analytics_20210602', tex_filename='SERC_HPC_analytics.tex',
#                                            groups=PI_groups, SACCT_obj=SACCT)


### CEES Sherlock report(s)
- Let's just get into this... What we need:
- Standard(ish) report of SERC (and other) partition(s) usage
- Mostly same standard report for each active PI group

#### Standard report:
- jobs,cpus
- cpu-hours
- Time-of-Day, Day-of-week usage
- 

#### Basic workflow:
- Get full SACCT data for `serc` partition
- Run report on partition usage
- Get active groups (group name + users?)
- Fetch new SACCT_obj for each group
- Run report for each group.

In [ ]:
#SACCT_obj.write_hdf5('sacct_data/serc_2021_12_01_2022_01_31.h5')

In [ ]:
# SACCT_obj = hpc_lib.SACCT_data_direct(partition='serc', start_date='2021-12-01', end_date='2022-01-31',
#                                   keep_raw_data=False)
# SACCT_obj.write_hdf5('sacct_data/serc_2021_12_01_2022_01_31.h5')

serc_report = hpc_lib.SACCT_groups_analyzer_report(Short_title='SERC Analytics', Full_title='SERC HPC analytics',
                                                  SACCT_obj=SACCT_obj,
                                                  out_path='output/SERC_analytics_20220121',
                                                   tex_filename='SERC_HPC_analytics.tex'
                                                  )



In [ ]:
# How to make reports with hpc_lib.SACCT_groups_analyzer_report_handler()
#
serc_report_h = hpc_lib.SACCT_groups_analyzer_report_handler(Short_title='SERC Analytics', Full_title='SERC HPC analytics',
                                                  SACCT_obj=SACCT_obj,
                                                  out_path='output/SERC_analytics_20220121_h',
                                                   tex_filename='SERC_HPC_analytics.tex'
                                                  )
z = serc_report_h.standard_reports_slides(group_name='serc')

#z = serc_report_h.standard_reports_slides(ix=(SACCT_obj.jobs_summary['Group']==b'biondo'), group_name='Biondo')

zz = serc_report_h.HPC_tex_obj.render()
#ix_g_ruthm = numpy.where([SACCT_obj.jobs_summary['User'] ])
#ix_g_ruthm = (SACCT_obj.jobs_summary['Group']=='ruthm')

In [ ]:
#SACCT_obj.

In [ ]:

#hourlies = SACCT_obj.get_cpu_hours(bin_size=1./24., d_t=1/24., IX=None, verbose=0)
# NOTE: because we have 1/24 d_t and bin_size, this should be equivalent to active_cpus, but it is not
#. actually computed quite the same way.
hourlies = hpc_lib.get_cpu_hours(bin_size=7., d_t=1/24., verbose=0, jobs_summary=SACCT_obj.jobs_summary)

#hourlies = hpc_lib.get_cpu_hours(bin_size=1./24., n_points=None, d_t=1./24., verbose=0, jobs_summary=SACCT_obj.jobs_summary)

print('** hourlies: ', hourlies['cpu_hours'][0:20]*24.)

In [ ]:
hourlies_cpu_job = hpc_lib.active_jobs_cpu(bin_size=1./24., jobs_summary=SACCT_obj.jobs_summary)
hourlies_cpu_job['N_jobs'][numpy.isnan(hourlies_cpu_job['N_jobs'])] = 0.
hourlies_cpu_job['N_cpu'][numpy.isnan(hourlies_cpu_job['N_cpu'])] = 0.

In [ ]:
print('** ', hourlies_cpu_job.dtype)
print(numpy.isnan(hourlies_cpu_job['N_jobs']).any() )
print(numpy.isnan(hourlies_cpu_job['N_cpu']).any() )
print(numpy.isnan(hourlies_cpu_job['time']).any() )

In [ ]:
# I don't know why these are not the same length... something to do with how cpu-hours are computed.
print('lens: ', len(hourlies), len(hourlies_cpu_job))

In [ ]:
# NOTE: num2date(0) is a thursday, so datetime.weekday() == (t+3)%7 or (t-4)%7, so t%7 = {2,3} are sat, sun.
ix_wknd   = numpy.logical_and( (hourlies['time'])%7 >= 2., (hourlies['time'])%7 <= 3. )
ix_afters = numpy.logical_or(hourlies['time']%1.<.3 , hourlies['time']%1>.8)
ix_dollies = numpy.logical_and(numpy.invert(ix_wknd), numpy.invert(ix_afters))

In [ ]:
ix_w   = numpy.logical_and( (hourlies_cpu_job['time'])%7 >= 2., (hourlies_cpu_job['time'])%7 <= 3. )
#ix_a   = numpy.logical_or(hourlies_cpu_job['time']%1.<.3 , hourlies_cpu_job['time']%1>.8)
ix_a   = numpy.logical_and( numpy.logical_or(hourlies_cpu_job['time']%1.<.33 , hourlies_cpu_job['time']%1>.75),
                           numpy.invert(ix_w))
ix_d   = numpy.logical_and(numpy.invert(ix_w), numpy.invert(ix_a))

In [ ]:
print('** ', hourlies['cpu_hours'][0:10])

In [ ]:
# print('*** ', SACCT_obj.jobs_summary[0:10])
# print('*** ', SACCT_obj.jobs_summary.dtype)
# print('*** ', SACCT_obj.jobs_summary['Start'][0:10])
#
# what do dates look like? Turns out that the SLURM dates are TZ naive, but in local (PST) time.
# so matplotlib just treats them like UTC, so our treatment should be fine...
dt_str = '2021-12-08T11:05:18'
dt_num = mpd.datestr2num(dt_str)
print(f'** dt_num: {dt_num}')

In [ ]:

# compute running averages over a full week for the components:
# dollies (9 to 5)
# after hours (weekdays, not 9 to 5)
# weekends.
ave_bins = 7*24
#
hourlies_cake_total = running_mean(hourlies_cpu_job['N_cpu'], ave_bins)
#
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_d)] = 0.
hourlies_cake_9to5 = running_mean(X,ave_bins)
#
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_w)] = 0.
hourlies_cake_wknd = running_mean(X, ave_bins)
#
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_a)] = 0.
hourlies_cake_afters = running_mean(X, ave_bins)
#

# print('** ', hourlies['cpu_hours'][0:10]*24.)
# print('** ', hourlies_cake_afters[0:100])

In [ ]:
dt = dtm.datetime.now()
dt_64 = numpy.datetime64(dt)
#
print('** ', dt.weekday())
print('** ', int((mpd.date2num(dt)+3)%7))
#
print('** ', isinstance(numpy.datetime64(dtm.datetime.now()), dtm.datetime))
#
print('*** ', mpd.date2num(dt), mpd.date2num(dt_64))
#
print('*** ', numpy.mod(numpy.arange(10),2) )

In [ ]:
#q_steps = numpy.array([.25, .5, .75, .9])
qs = numpy.array([.25, .5, .75, .9])
print('** ', numpy.sum(hourlies_cpu_job['N_cpu']), numpy.sum(hourlies_cpu_job['N_cpu'][ix_d]),
      numpy.sum(hourlies_cpu_job['N_cpu'][ix_a]), numpy.sum(hourlies_cpu_job['N_cpu'][ix_w]))
#
dtype_qs = [('time', '>i8')] + [(f'q{k+1}', '>f8') for k,x in enumerate(qs)] 
print('** dtype: ', dtype_qs)
periodic_hourly_weekdays_cpus = numpy.zeros((24,), dtype=dtype_qs)
periodic_hourly_weekdays_cpus['time'] = numpy.arange(24)
print('** ', periodic_hourly_weekdays_cpus)
print('** ', periodic_hourly_weekdays_cpus['time'])
#
for k in periodic_hourly_weekdays_cpus['time']:
    # we are really interested in 9to5 activity, but since we're resolving hours, we might as well show 
    #. them. SO, we want to just exclude weekends, then do quantiles on daily slices.
    q_vals = numpy.nanquantile(hourlies_cpu_job['N_cpu'][numpy.logical_and(numpy.invert(ix_w),
                                    (24*(hourlies_cpu_job['time']%1)).astype(int)==k)], qs)
    #print(f'** qs[{k}]: {q_vals}')
    #
    # what is this syntax????
    #periodic_hourly_925_cpus[('q1', 'q2', 'q3')] = q_vals[:]
    #if numpy.isnan(q_vals):
    #    continue
    for j,q in enumerate(q_vals):
        periodic_hourly_weekdays_cpus[f'q{j+1}'][k]=q
print('** **\n', periodic_hourly_weekdays_cpus)

periodic_daily_925_cpus = numpy.zeros((7,), dtype=dtype_qs)
periodic_daily_cpus = numpy.zeros((7,), dtype=dtype_qs)
periodic_daily_925_cpus['time'] = numpy.arange(7)
periodic_daily_cpus['time'] = numpy.arange(7)
#
# daily usage:
for k in periodic_daily_cpus['time']:
    # here, we want do do quantiles on daily slices. Again, our principal interest is
    #. 9to5 activity, so exclude after-hours (t_am > t < t_pm), but include weekends. 
#    q_vals_925 = numpy.nanquantile(hourlies_cpu_job['N_cpu'][numpy.logical_and(numpy.invert(ix_d),
#                                    (((hourlies_cpu_job['time']+3)%7)).astype(int)==k)], qs)
    # ix_a   = numpy.logical_and( numpy.logical_or(hourlies_cpu_job['time']%1.<.33 , hourlies_cpu_job['time']%1>.75),
    #                       numpy.invert(ix_w))
    ix_925 = numpy.logical_and(numpy.logical_and((hourlies_cpu_job['time']%1. >= .33),
                                             (hourlies_cpu_job['time']%1 <= .75)),
                           (((hourlies_cpu_job['time']+3)%7).astype(int)==k)
                          )
    #X = hourlies_cpu_job['N_cpu'][ix]
    #print('*** sum(ix): ', numpy.sum(ix) )
    q_vals_925 = numpy.nanquantile(hourlies_cpu_job['N_cpu'][ix_925], qs)
    #
#    q_vals     = numpy.nanquantile(hourlies_cpu_job['N_cpu'][numpy.logical_and(numpy.invert(ix_a),
#                                    (((hourlies_cpu_job['time']+3)%7)).astype(int)==k)], qs)
    q_vals     = numpy.nanquantile(hourlies_cpu_job['N_cpu'][(((hourlies_cpu_job['time']+3)%7)).astype(int)==k], qs)

    #                                                             
    #print(f'** ** q_vals_925[{k}]: {q_vals_925}')
    for j,q in enumerate(q_vals):
        periodic_daily_cpus[f'q{j+1}'][k]=q
    for j,q in enumerate(q_vals_925):
        periodic_daily_925_cpus[f'q{j+1}'][k]=q

In [ ]:
#print('** ', ix_dollies[0:100])
fg = plt.figure(figsize=(14,15))
n_cols = 2
n_rws  = 4
ax1 = plt.subplot(n_rws,1,1)
ax2 = plt.subplot(n_rws,1,2, sharex=ax1)
ax3 = plt.subplot(n_rws,n_cols,5)
ax4 = plt.subplot(n_rws,n_cols,6)
ax5 = plt.subplot(n_rws,n_cols,7, projection='polar')
ax6 = plt.subplot(n_rws,n_cols,8, projection='polar')
#
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax5.grid()
ax6.grid()
#
fg.suptitle('CPU Usage Report', size=16)
ax1.set_title('Active CPUs')
ax2.set_title('Active CPUs layer cake')
ax3.set_title('CPUs Active: Hourly, weekdays')
ax4.set_title('CPUs Active: DoW, 9-5')
#
# ix=numpy.logical_and( T%1.>.3 , T%1<.8, (T-4)%7)
# ax.plot(hourlies['time'], hourlies['cpu_hours'])
# ax.plot(hourlies['time'][ix], hourlies['cpu_hours'][ix], ls='', marker='.')
ax1.plot(hourlies_cpu_job['time'], hourlies_cpu_job['N_cpu'], color='b', lw=1.)
#ax1.fill_between(hourlies_cpu_job['time'], 0., hourlies_cpu_job['N_cpu'], color='b', alpha=.2, zorder=1)
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_d)]=0.
ax1.fill_between(hourlies_cpu_job['time'], 0., X, alpha=.2, label='9 to 5')
#
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_w)]=0.
ax1.fill_between(hourlies_cpu_job['time'], 0., X, alpha=.2, label='weekends')
#
X = hourlies_cpu_job['N_cpu'].copy()
X[numpy.invert(ix_a)]=0.
X[ix_w]=0.
ax1.fill_between(hourlies_cpu_job['time'], 0., X, alpha=.2, label='after-hours')
# x_lbls = [mpd.num2date(x) for x in ax1.get_xticklabels.text]
# ax1.set_xticklabels([f'{x.year}-{x.month}-{x.day}' for x in x_lbls])
ax1.legend(loc=0)
#

# ax1.plot(hourlies_cpu_job['time'][ix_d], hourlies_cpu_job['N_cpu'][ix_d], ls='', marker='.')
# ax1.plot(hourlies_cpu_job['time'][ix_w], hourlies_cpu_job['N_cpu'][ix_w], ls='', marker='.')
# ax1.plot(hourlies_cpu_job['time'][ix_a], hourlies_cpu_job['N_cpu'][ix_a], ls='', marker='.')
#
T = hourlies_cpu_job['time']
ln, = ax2.plot(T[ave_bins-1:], hourlies_cake_total, ls='-', color='r', label='total')
#ax2.plot(T, hourlies['cpu_hours'], ls='-', marker='')
X = hourlies_cake_wknd.copy()
ln, = ax2.plot(T[ave_bins-1:], X, ls='--', label='weekends')
clr = ln.get_color()
ax2.fill_between(T[ave_bins-1:], 0., X, color=clr, alpha=.2 )
#
X0 = X.copy()
X += hourlies_cake_afters
ln, = ax2.plot(T[ave_bins-1:], X, ls='--', label='after-hours')
clr = ln.get_color()
ax2.fill_between(T[ave_bins-1:], X0, X, color=clr, alpha=.2 )
#
X0 = X.copy()
X += hourlies_cake_9to5
ln, = ax2.plot(T[ave_bins-1:], X, ls='--', marker='', label='9to5')
clr = ln.get_color()
ax2.fill_between(T[ave_bins-1:], X0, X, color=clr, alpha=.2 )
#
ax2.legend(loc=0)
fg.canvas.draw()
#print('** ** DEBUG: ', [s.get_text() for s in ax2.get_xticklabels()])
lbls = [hpc_lib.simple_date_string(mpd.num2date(max(1, float(s.get_text())) ) ) for s in ax2.get_xticklabels()]
ax2.set_xticklabels(lbls)
#
ln, = ax3.plot(periodic_hourly_weekdays_cpus['time'], periodic_hourly_weekdays_cpus['q2'], lw=3)
clr = ln.get_color()
ax3.fill_between(periodic_hourly_weekdays_cpus['time'], periodic_hourly_weekdays_cpus['q2'],
                 periodic_hourly_weekdays_cpus['q3'], color=clr, alpha=.15 )
ax3.fill_between(periodic_hourly_weekdays_cpus['time'], periodic_hourly_weekdays_cpus['q2'],
                 periodic_hourly_weekdays_cpus['q4'], color=clr, alpha=.15 )

xx = numpy.min(numpy.append([1000], periodic_hourly_weekdays_cpus['q1']))
ax3.set_ylim(xx, 5000)
#
X = periodic_daily_925_cpus.copy()
ax4.plot(X['time'], X['q2'], lw=3, color='b', label='9to5')
ax4.fill_between(X['time'], X['q2'], X['q3'], color='b', alpha=.15 )
ax4.fill_between(X['time'], X['q1'], X['q4'], color='b', alpha=.15 )
X = periodic_daily_cpus
ax4.plot(X['time'], X['q1'], lw=3, color='m', ls='--', label='all')
ax4.fill_between(X['time'], X['q2'], X['q4'], color='m', alpha=.08)
ax4.legend(loc=0)
#
ax5.set_theta_direction(-1)
ax5.set_theta_offset(math.pi/2.0)
X = periodic_hourly_weekdays_cpus['time'].copy()*scipy.constants.pi*2.0/float(len(periodic_hourly_weekdays_cpus))
ln, = ax5.plot(X, periodic_hourly_weekdays_cpus['q2'], lw=3, marker='.')
clr = ln.get_color()
ax5.plot(numpy.linspace(0., math.pi*2., 100), numpy.ones(100)*numpy.mean(periodic_hourly_weekdays_cpus['q2']),
         ls='--', lw=2., color=clr, alpha=.7, label='mean, 50th' )
ax5.set_xticks(numpy.arange(0., math.pi*2., math.pi*2/24.))
ax5.set_xticklabels(numpy.arange(0,24, 1.))
y_min = .5*(numpy.max(periodic_hourly_weekdays_cpus['q1'])+numpy.min(periodic_hourly_weekdays_cpus['q2']))
ax5.set_ylim(y_min, )
ax5.grid()
ax5.legend(loc='upper right')
#
# TODO: figure out phase of DoW circle-plot.
ax6.set_theta_direction(-1)
ax6.set_theta_offset(math.pi/2.0)
X = periodic_daily_925_cpus.copy()
ln, = ax6.plot(X['time']*math.pi*2.0/float(len(X)+1.), X['q2'], lw=3, marker='o', label='9to5')
clr = ln.get_color()
ax6.plot(numpy.linspace(0., math.pi*2., 100), numpy.ones(100)*numpy.mean(X['q2']), ls='--', lw=2., 
         color=clr, alpha=.7 , label='mean, 50th')
ax6.set_xticks=numpy.arange(0., math.pi*2., math.pi*2./7.)
ax6.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
y_min = .5*(numpy.max(X['q1']) + numpy.min(X['q2']))
ax6.set_ylim(y_min, )
ax6.grid()
ax6.legend(loc='upper left')


In [ ]:
print('** ', hourlies['cpu_hours'][0:10])
print('** ', hourlies_cake_9to5[0:10])



In [ ]:
# for t in hourlies['time'][0:250]:
#     d = mpd.num2date(t)
#     print('** time: {}: hour: {}, weekday: {}/{}/{}::{}'.format(t, d.hour, d.weekday(), int(t%7), 
#                                                                 int(t+3)%7, int(t-4)%7 ))

In [ ]:
#
out_path = 'output/SERC_analytics'
#
rpt = hpc_reports.SACCT_report_handler(SACCT_obj=SACCT_obj, out_path='output/SERC_analytics')
#
fpath_cpu_activity_serc = rpt.cpu_hourly_activity_report(fout_path_name='')
fpath_wait_stats = rpt.wait_stats_report(fout_path_name='')
#
# TODO: need to get relative paths correct here. render() defaults to render from the the loal path.
#. use os.path.relpath(path_to, path_from) and os.lpath.abspath() to code the paths into the 
#. report definition.
rpt.HPC_tex_obj.add_fig_slide(fig_title='CPU Activity: SERC', fig_path=fpath_cpu_activity_serc)
rpt.HPC_tex_obj.add_fig_slide(fig_title='Wait Stats: SERC', fig_path=fpath_wait_stats)
#

In [ ]:
print('*** ', fpath_cpu_activity_serc, os.path.abspath(fpath_cpu_activity_serc))
print('*** ', fpath_wait_stats)
print('*** ', os.path.join('abc', 'def', 'ghi'))

In [ ]:
rpt.HPC_tex_obj.render()
#rpt.HPC_tex_obj.save_presentation_tex(os.path.join(out_path, 'SERC_usage.json'))


#### Wait Stats figure/report:
- Do a wait-time timeseries (just to see how it looks), maybe with some averaging.
- Wait time quantiles, again by day, hour, 9to5, and for a few ranges of n_cpu.

In [ ]:
# wait_stats      = SACCT_obj.get_wait_stats(qs=[.25, .5, .75, .9])
# for col in wait_stats.dtype.names[1:]:
#     wait_stats[col]*=24.
#wait_stats_ncpu = SACCT_obj.get_wait_times_per_ncpu()

wait_times = 60.*24.*(SACCT_obj.jobs_summary['Start'] - SACCT_obj.jobs_summary['Submit'])
# wait_time_units = 'hours'
# wait_times *= 60.
wait_time_units='minutes'
#

wait_times_per_dow = hpc_lib.day_of_week_distribution(time=SACCT_obj.jobs_summary['Submit'], Y=wait_times)
wait_times_per_hod = hpc_lib.hour_of_day_distribution(time=SACCT_obj.jobs_summary['Submit'], Y=wait_times)

#
qs_ps = [.25, .5, .76, .9]
qs_waittimes = numpy.nanquantile(wait_times,qs_ps)
print('*** Wait-time quantiles: {}'.format(qs_waittimes))
ix_q4 = numpy.logical_and(numpy.isnan(wait_times)==False, wait_times<qs_waittimes[-1])
#

In [ ]:
# Wait stats on serc are basically nothing. Let's see if we can get something interesting for
# ncpus: <=10, 10<n<100, n>100\
#
# kx_wait_10 = SACCT_obj.jobs_summary['NCPUS']<=10
# kx_wait_100 = numpy.logical_and(SACCT_obj.jobs_summary['NCPUS']>10,  SACCT_obj.jobs_summary['NCPUS']<=100)
# kx_wait_101 = SACCT_obj.jobs_summary['NCPUS']>100
#
fg = plt.figure(figsize=(12,10))
ax1a = plt.subplot(3,2,1, projection='polar')
ax1a.set_theta_direction(-1)
ax1a.set_theta_offset(math.pi/2.0)
ax1a.set_title('Wait-Time DoW quantiles (hours)')
#
ax1b = plt.subplot(3,2,2, projection='polar')
ax1b.set_theta_direction(-1)
ax1b.set_theta_offset(math.pi/2.0)
ax1b.set_title('Wait-Time ToD quantiles (hours)')
#
ax2a = plt.subplot(3,2,3)
ax2a.set_title('Wait-Time DoW (clock plot)')
ax2b = plt.subplot(3,2,4)
ax2b.set_title('Wait-Time ToD (clock plit)')

ax3 = plt.subplot(3,2,5)
ax4 = plt.subplot(3,2,6)

#
X_dow = wait_times_per_dow['DoW']*math.pi*2./7.
ln, = ax1a.plot(X_dow, wait_times_per_dow['q2'], ls='-', marker='o')
clr = ln.get_color()
ax1a.fill_between(X_dow, wait_times_per_dow['q2'], wait_times_per_dow['q3'], color=clr, alpha=.2)
ax1a.set_xticks(X_dow)
ax1a.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
#
X_hod = wait_times_per_hod['hour']*math.pi*2./float(len(wait_times_per_hod))
ln, = ax1b.plot(X_hod, wait_times_per_hod['q2'], ls='-', marker='o')
clr = ln.get_color()
ax1b.fill_between(X_hod, wait_times_per_hod['q2'], wait_times_per_hod['q3'], color=clr, alpha=.2)
ax1b.set_xticks(X_hod )
ax1b.set_xticklabels(numpy.arange(0,24,1))
#
X_dow = wait_times_per_dow['DoW']
ln, = ax2a.plot(X_dow, wait_times_per_dow['q2'], ls='-', marker='o')
clr = ln.get_color()
ax2a.fill_between(X_dow, wait_times_per_dow['q2'], wait_times_per_dow['q3'], color=clr, alpha=.2)
ax2a.set_xticks(numpy.arange(0, 7) )
ax2a.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
ax2a.set_ylabel(wait_time_units.capitalize())
#
X_hod = wait_times_per_hod['hour']
ln, = ax2b.plot(X_hod, wait_times_per_hod['q2'], ls='-', marker='o')
clr = ln.get_color()
ax2b.fill_between(X_hod, wait_times_per_hod['q2'], wait_times_per_hod['q3'], color=clr, alpha=.2)

qs_wait = numpy.quantile(wait_times[numpy.invert(numpy.isnan(wait_times))], qs_ps)
print('*** ', qs_wait)
print('*** wait time quantiles: ')
for q_p, q_v in zip(qs_ps, qs_wait):
    print(f'** q[{q_p}]: {q_v*60.} ')
print('*** mean: {}'.format(numpy.mean(wait_times[numpy.invert(numpy.isnan(wait_times))])))
#

h1 = ax3.hist(wait_times[ix_q4], bins=1000, density=True)
#ax3.set_xlim(0., 1.)
ax3.set_title(f'Wait time PDF of {qs_ps[3]:.2f} quantile')
ax3.set_xlabel(wait_time_units.capitalize())
ax3.set_ylabel('Percent')
#
h2 = ax4.hist(wait_times[ix_q4], bins=100, cumulative=True, density=True, histtype='step')
h2 = ax4.hist(wait_times[ix_q4], bins=100, cumulative=True, density=True, histtype='stepfilled', alpha=.2)
#ax4.set_xlim(0, 1.)
ax4.set_title(f'Wait time CDF of {qs_ps[3]:.2f} quantile')
ax4.set_xlabel(wait_time_units.capitalize())
ax4.set_ylabel('Percent')
#
for ax in (ax2a, ax2b, ax3, ax4):
    ax.grid()



In [ ]:
# let's just write explicit functions for periodic usage of cpus, jobs, other-stuff-too?
# We could use the existing tools to modulus-bin time-series, but I think it won't cost that much
# to just do it explicitly and (more) exactly. Trying to match up

In [ ]:

print('** ', wait_times[0:15])

In [ ]:
print('** ', wait_stats[0:10])
print('** ', wait_stats.dtype.names)
print('** ', wait_stats.qs)